Unigram

In [231]:
#!/bin/python

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import collections
from math import log
import sys
from nltk.util import ngrams

# Python 3 backwards compatibility tricks
if sys.version_info.major > 2:

    def xrange(*args, **kwargs):
        return iter(range(*args, **kwargs))

    def unicode(*args, **kwargs):
        return str(*args, **kwargs)

class LangModel:
    def fit_corpus(self, corpus):
        """Learn the language model for the whole corpus.

        The corpus consists of a list of sentences."""
        for s in corpus:
            self.fit_sentence(s)
            
       # a = Counter(self.model.values())
        #a = sorted(a.items(), key=lambda x: x[1])
        #print("words count occurance",a)
        vo = self.vocab()
        poplist = list()
        for w in vo:
            if self.model[w] == 1:
                poplist.append(w)
        self.model[('UNK',)] = len(poplist)
        print("total UNK is ", len(poplist))
        for p in poplist:
            self.model.pop(p)
        self.norm()
    def perplexity(self, corpus):
        """Computes the perplexity of the corpus by the model.

        Assumes the model uses an EOS symbol at the end of each sentence.
        """
        return pow(2.0, self.entropy(corpus))

    def entropy(self, corpus):
        num_words = 0.0
        sum_logprob = 0.0
        for s in corpus:
            num_words += len(s) + 1 # for EOS
            sum_logprob += self.logprob_sentence(s)
        return -(1.0/num_words)*(sum_logprob)

    def logprob_sentence(self, sentence):
        p = 0.0
        unigrams = list(ngrams(sentence, 1, pad_left = False, pad_right=False))
        for i in xrange(len(sentence)):
            p += self.cond_logprob(unigrams[i],unigrams[:i])
        #p += self.cond_logprob('END_OF_SENTENCE', unigrams)
        return p

    # required, update the model when a sentence is observed
    def fit_sentence(self, sentence): pass
    # optional, if there are any post-training steps (such as normalizing probabilities)
    def norm(self): pass
    # required, return the log2 of the conditional prob of word, given previous words
    def cond_logprob(self, word, previous): pass
    # required, the list of words the language model suports (including EOS)
    def vocab(self): pass

class Trigram(LangModel):
    def __init__(self, backoff = 0.000001):
        self.modeltri = dict()
        self.modelbi = dict()
        self.model = dict()
        self.lbackoff = log(backoff, 2)
        
    def inc_word(self, w):
        opt = len(w)
        if opt == 1:
            if w in self.model:
                self.model[w] += 1.0
            else:
                self.model[w] = 1.0
        elif opt == 2:
            if w in self.modelbi:
                self.modelbi[w] += 1.0
            else:
                self.modelbi[w] = 1.0
        elif opt == 3:
            if w in self.modeltri:
                self.modeltri[w] += 1.0
            else:
                self.modeltri[w] = 1.0
        
        
    def fit_sentence(self, sentence):
        unigrams = list(ngrams(sentence, 1, pad_left = False, pad_right=False))
        bigrams = list(ngrams(sentence, 2, pad_left = True, left_pad_symbol= '*', pad_right=True,right_pad_symbol='END_OF_SENTENCE'))
        trigrams = list(ngrams(sentence, 3, pad_left = True, left_pad_symbol= '*', pad_right=True,right_pad_symbol='END_OF_SENTENCE'))
        trigrams = trigrams[:len(trigrams)-1]
        for w in unigrams:
            self.inc_word(w)
        for w in bigrams:
            self.inc_word(w)
        for w in trigrams:
            self.inc_word(w)
        
    def norm(self):
        """Normalize and convert to log2-probs."""
        tot = 0.0
        for word in self.model:
            tot += self.model[word]
        ltot = log(tot, 2)
        for word in self.model:
            self.model[word] = log(self.model[word], 2) - ltot

    def cond_logprob(self, word, previous):
        if word in self.model:
            return self.model[word]
        else:
            return self.model[('UNK',)]

    def vocab(self):
        return self.model.keys()

Trigram with laplace smoothing

In [361]:
#!/bin/python

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import collections
from math import log
import sys
from nltk.util import ngrams

# Python 3 backwards compatibility tricks
if sys.version_info.major > 2:

    def xrange(*args, **kwargs):
        return iter(range(*args, **kwargs))

    def unicode(*args, **kwargs):
        return str(*args, **kwargs)

class LangModel:
    def fit_corpus(self, corpus):
        """Learn the language model for the whole corpus.

        The corpus consists of a list of sentences."""
        for s in corpus:
            self.fit_sentence(s)
         
        vo = self.model.keys()
        vobi = self.modelbi.keys()
        votri = self.modeltri.keys()
        for w in vo:
            if self.model[w] == 1:
                self.poplist.append(w[0])
                
        self.fimodel[('UNK',)] = len(self.poplist)
        print("total UNK is ", len(self.poplist))
        stop = 0
        for tri in votri:
            #stop += 1
            #if stop == 10000:
             #   ('a','b')[0] = 'e'
            old = tri
            tri = list(tri)
            for i in range(0,len(tri)):
                if tri[i] in self.poplist:
                    tri[i] = 'UNK'
            tri = tuple(tri)
            if tri in self.fimodel:
                    self.fimodel[tri] += self.modeltri[old]
            else:
                    self.fimodel[tri] = self.modeltri[old]
        print("finished processing tri with unk")    
        for bi in vobi:
            oldbi = bi
            bi = list(bi)
            for i in range(0,len(bi)):
                if bi[i] in self.poplist:
                    bi[i] = 'UNK'
            bi = tuple(bi)
            if bi in self.fimodelbi:
                self.fimodelbi[bi] += self.modelbi[oldbi]
            else:
                self.fimodelbi[bi] = self.modelbi[oldbi]   
        print("finished processing bi with unk")    
        
        self.norm(corpus)

    def perplexity(self, corpus):
        """Computes the perplexity of the corpus by the model.

        Assumes the model uses an EOS symbol at the end of each sentence.
        """
        print('---------',self.pro,'------------')
        return pow(2.0, self.entropy(corpus))

    def entropy(self, corpus):
        num_words = 0.0
        sum_logprob = 0.0
        for s in corpus:
            num_words += len(s)  # for EOS
            sum_logprob += self.logprob_sentence(s,corpus)
        return -(1.0/num_words)*(sum_logprob)

    def logprob_sentence(self, sentence,corpus):
        p = 0.0
        trigrams = list(ngrams(sentence, 3, pad_left = True, left_pad_symbol= '*', pad_right=True,right_pad_symbol='END_OF_SENTENCE'))
        trigrams = trigrams[:len(trigrams)-1]
        for i in xrange(len(sentence)):
            p += self.cond_logprob(trigrams[i],trigrams[:i])
        return p

    # required, update the model when a sentence is observed
    def fit_sentence(self, sentence): pass
    # optional, if there are any post-training steps (such as normalizing probabilities)
    def norm(self,corpus): pass
    # required, return the log2 of the conditional prob of word, given previous words
    def cond_logprob(self, word, previous): pass
    # required, the list of words the language model suports (including EOS)
    def vocab(self): pass

class Trigram(LangModel):
    def __init__(self, backoff = 0.000001):
        self.modeltri = dict()
        self.modelbi = dict()
        self.model = dict()
        self.fimodel = dict()
        self.fimodelbi = dict()
        self.lbackoff = log(backoff, 2)
        self.pro = list()
        self.corpus = list()
        self.poplist = list()
    def inc_word(self, w):
        opt = len(w)
        if opt == 1:
            if w in self.model:
                self.model[w] += 1.0
            else:
                self.model[w] = 1.0
        elif opt == 2:
            if w in self.modelbi:
                self.modelbi[w] += 1.0
            else:
                self.modelbi[w] = 1.0
        elif opt == 3:
            if w in self.modeltri:
                self.modeltri[w] += 1.0
            else:
                self.modeltri[w] = 1.0
        
        
    def fit_sentence(self, sentence):
        unigrams = list(ngrams(sentence, 1, pad_left = False, pad_right=False))
        bigrams = list(ngrams(sentence, 2, pad_left = True, left_pad_symbol= '*', pad_right=True,right_pad_symbol='END_OF_SENTENCE'))
        trigrams = list(ngrams(sentence, 3, pad_left = True, left_pad_symbol= '*', pad_right=True,right_pad_symbol='END_OF_SENTENCE'))
        trigrams = trigrams[:len(trigrams)-1]
        for w in unigrams:
            self.inc_word(('END_OF_SENTENCE',))
            self.inc_word(w)
        for w in bigrams:
            self.inc_word(w)
        for w in trigrams:
            self.inc_word(w)
       
    def norm(self,corpus):
        """Normalize and convert to log2-probs."""
        v = 0.01*len(self.model.keys())
        binum = 0
        for k in self.fimodelbi:
            if "END_OF_SENTENCE" not in k:
                binum += 1
        for word in self.fimodel:
        #for word in self.modeltri:
            if word == ('UNK',):
                tot = 0
                for w in self.model:
                    tot += self.model[w]
                print("tot is ", tot, "v is",v)
                self.fimodel[word] = log(0.1* self.fimodel[word], 2) - log(v+tot, 2)

            elif (word[0],word[1]) == ('*','*') :
                self.fimodel[word] = log(0.01+self.fimodel[word],2)-log(v+len(corpus),2)
            else:
                self.fimodel[word] = log(0.01+self.fimodel[word], 2) -log(v+self.fimodelbi[(word[0],word[1])],2) 
                
   
    def cond_logprob(self, word, previous):
        uni = self.model.keys()
        word = list(word)
        for i in range(0,len(word)):
            if (word[i] in self.poplist) or ((word[i],) not in uni):
                word[i] = 'UNK'
        word = tuple(word)
        if word in self.fimodel:
            return self.fimodel[word]
  
        else:
            v = len(self.model.keys())
            return self.fimodel[('UNK',)]

    def vocab(self):
        return self.fimodel.keys()

In [410]:
#!/bin/python

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import numpy as np
import sys


# Python 3 backwards compatibility tricks
if sys.version_info.major > 2:

    def xrange(*args, **kwargs):
        return iter(range(*args, **kwargs))

    def unicode(*args, **kwargs):
        return str(*args, **kwargs)


def textToTokens(text):
    """Converts input string to a corpus of tokenized sentences.

    Assumes that the sentences are divided by newlines (but will ignore empty sentences).
    You can use this to try out your own datasets, but is not needed for reading the homework data.
    """
    corpus = []
    sents = text.split("\n")
    from sklearn.feature_extraction.text import CountVectorizer
    count_vect = CountVectorizer()
    count_vect.fit(sents)
    tokenizer = count_vect.build_tokenizer()
    for s in sents:
        toks = tokenizer(s)
        if len(toks) > 0:
            corpus.append(toks)
    return corpus

def file_splitter(filename, seed = 0, train_prop = 0.7, dev_prop = 0.15,
    test_prop = 0.15):
    """Splits the lines of a file into 3 output files."""
    import random
    rnd = random.Random(seed)
    basename = filename[:-4]
    train_file = open(basename + ".train.txt", "w")
    test_file = open(basename + ".test.txt", "w")
    dev_file = open(basename + ".dev.txt", "w")
    with open(filename, 'r') as f:
        for l in f.readlines():
            p = rnd.random()
            if p < train_prop:
                train_file.write(l)
            elif p < train_prop + dev_prop:
                dev_file.write(l)
            else:
                test_file.write(l)
    train_file.close()
    test_file.close()
    dev_file.close()

def read_texts(tarfname, dname):
    """Read the data from the homework data file.

    Given the location of the data archive file and the name of the
    dataset (one of brown, reuters, or gutenberg), this returns a
    data object containing train, test, and dev data. Each is a list
    of sentences, where each sentence is a sequence of tokens.
    """
    import tarfile
    tar = tarfile.open(tarfname, "r:gz", errors = 'replace')
    for member in tar.getmembers():
        if dname in member.name and ('train.txt') in member.name:
            print('\ttrain: %s'%(member.name))
            train_txt = unicode(tar.extractfile(member).read(), errors='replace')
        elif dname in member.name and ('test.txt') in member.name:
            print('\ttest: %s'%(member.name))
            test_txt = unicode(tar.extractfile(member).read(), errors='replace')
        elif dname in member.name and ('dev.txt') in member.name:
            print('\tdev: %s'%(member.name))
            dev_txt = unicode(tar.extractfile(member).read(), errors='replace')

    from sklearn.feature_extraction.text import CountVectorizer
    count_vect = CountVectorizer()
    count_vect.fit(train_txt.split("\n"))
    tokenizer = count_vect.build_tokenizer()
    class Data: pass
    data = Data()
    data.train = []
    for s in train_txt.split("\n"):
        toks = tokenizer(s)
        if len(toks) > 0:
            data.train.append(toks)
    data.test = []
    for s in test_txt.split("\n"):
        toks = tokenizer(s)
        if len(toks) > 0:
            data.test.append(toks)
    data.dev = []
    for s in dev_txt.split("\n"):
        toks = tokenizer(s)
        if len(toks) > 0:
            data.dev.append(toks)
    print(dname," read.", "train:", len(data.train), "dev:", len(data.dev), "test:", len(data.test))
    return data

def learn_trigram(data, verbose=True):
    """Learns a trigram model from data.train.

    It also evaluates the model on data.dev and data.test, along with generating
    some sample sentences from the model.
    """
    #from lm import Trigram
    trigram = Trigram()
    trigram.fit_corpus(data.train)
    sumpro = 0;
    #for w in trigram.model:
     #   sumpro += 2 ** (trigram.model[w])
    
    #for w in trigram.modeltri:
    #    sumpro += 2 ** (trigram.modeltri[w])
    #for k in trigram.modelbi:
    #    if "END_OF_SENTENCE" not in k:
    #        binum += 1
            
    #for w in trigram.fimodel:
     #   sumpro += 2 ** (trigram.fimodel[w])
   
    #print("sum of p(w):", sumpro)
    
    if verbose:
        print("vocab:", len(trigram.vocab()))
        # evaluate on train, test, and dev
        print("train:", trigram.perplexity(data.train))
        print("dev  :", trigram.perplexity(data.dev))
        print("test :", trigram.perplexity(data.test))
        sampler = Sampler(trigram)
        print("sample 1: ", " ".join(str(x) for x in sampler.sample_sentence([])))
        print("sample 2: ", " ".join(str(x) for x in sampler.sample_sentence([])))
    
    return trigram

def print_table(table, row_names, col_names, latex_file = None):
    """Pretty prints the table given the table, and row and col names.

    If a latex_file is provided (and tabulate is installed), it also writes a
    file containing the LaTeX source of the table (which you can \input into your report)
    """
    try:
        from tabulate import tabulate
        row_format ="{:>15} " * (len(col_names) + 1)
        rows = map(lambda rt: [rt[0]] + rt[1], zip(row_names,table.tolist()))

        print(tabulate(rows, headers = [""] + col_names))
        if latex_file is not None:
            latex_str = tabulate(rows, headers = [""] + col_names, tablefmt="latex")
            with open(latex_file, 'w') as f:
                f.write(latex_str)
                f.close()
    except ImportError as e:
        for row_name, row in zip(row_names, table):
            print(row_format.format(row_name, *row))

In [414]:
#!/bin/python

#from lm import LangModel
import random
from math import log
import numpy as np

class Sampler:

    def __init__(self, lm, temp = 0.6):
        """Sampler for a given language model.

        Supports the use of temperature, i.e. how peaky we want to treat the
        distribution as. Temperature of 1 means no change, temperature <1 means
        less randomness (samples high probability words even more), and temp>1
        means more randomness (samples low prob words more than otherwise). See
        simulated annealing for what this means.
        """
        self.lm = lm
        self.rnd = random.Random()
        self.temp = temp

    def sample_sentence(self, prefix = [], max_length = 8):
        """Sample a random sentence (list of words) from the language model.

        Samples words till either EOS symbol is sampled or max_length is reached.
        Does not make any assumptions about the length of the context.
        """
        i = 0
        sent = prefix
        word = self.sample_next(sent, False)
        while i <= max_length and "END_OF_SENTENCE" not in word:
            for j in range(0,len(word)):
                sent.append(word[j])
            word = self.sample_next(sent)
            i += 1
        return sent

    def sample_next(self, prev, incl_eos = True):
        """Samples a single word from context.

        Can be useful to debug the model, for example if you have a bigram model,
        and know the probability of X-Y should be really high, you can run
        sample_next([Y]) to see how often X get generated.

        incl_eos determines whether the space of words should include EOS or not.
        """
        wps = []
        tot = -np.inf # this is the log (total mass)
        for w in self.lm.vocab():
            if not incl_eos and "END_OF_SENTENCE" in w:
                continue
            if w[0] == '*':
                continue
            lp = self.lm.cond_logprob(w, prev)
            wps.append([w, lp/self.temp])
            tot = np.logaddexp2(lp/self.temp, tot)
        p = self.rnd.random()
        word = self.rnd.choice(wps)[0]
        s = -np.inf # running mass
        for w,lp in wps:
            s = np.logaddexp2(s, lp)
            if p < pow(2, s-tot):
                word = w
                break
        return word

trigram = Trigram()
corpus = [
    data.train[0],data.train[1],data.train[0],data.train[0]
]
trigram.fit_corpus(corpus)
sampler = Sampler(trigram)
for i in xrange(5):
    print(i, ":", " ".join(str(x) for x in sampler.sample_sentence([])))

total UNK is  29
finished processing tri with unk
finished processing bi with unk
0 : UNK and UNK to UNK UNK of the UNK the UNK UNK UNK to UNK UNK with UNK the UNK UNK with UNK UNK of the UNK
1 : UNK the UNK of the UNK UNK with UNK to UNK UNK UNK and UNK UNK to UNK to UNK UNK with UNK UNK and UNK UNK
2 : with UNK UNK with UNK UNK with UNK UNK to UNK UNK of the UNK with UNK UNK to UNK UNK of the UNK with UNK UNK
3 : of UNK and UNK the UNK of the UNK UNK to UNK UNK and UNK to UNK UNK UNK and UNK UNK UNK UNK to UNK UNK
4 : UNK with UNK UNK and UNK with UNK UNK with UNK UNK to UNK UNK of the UNK of UNK and UNK and UNK UNK UNK the


Run Unigram

In [247]:
from collections import Counter
dnames = [ "brown","reuters", "gutenberg"]
datas = []
models = []
# Learn the models for each of the domains, and evaluate it
for dname in dnames:
    print("-----------------------")
    print(dname)
    data = read_texts("data/corpora.tar.gz", dname)
    datas.append(data)
    model = learn_trigram(data)
    models.append(model)
# compute the perplexity of all pairs
n = len(dnames)
perp_dev = np.zeros((n,n))
perp_test = np.zeros((n,n))
perp_train = np.zeros((n,n))
for i in xrange(n):
    for j in xrange(n):
        perp_dev[i][j] = models[i].perplexity(datas[j].dev)
        perp_test[i][j] = models[i].perplexity(datas[j].test)
        perp_train[i][j] = models[i].perplexity(datas[j].train)

print("-------------------------------")
print("x train")
print_table(perp_train, dnames, dnames, "table-train.tex")
print("-------------------------------")
print("x dev")
print_table(perp_dev, dnames, dnames, "table-dev.tex")
print("-------------------------------")
print("x test")
print_table(perp_test, dnames, dnames, "table-test.tex")

-----------------------
brown
	test: brown/brown.test.txt
	train: brown/brown.train.txt
	dev: brown/brown.dev.txt
brown  read. train: 39802 dev: 8437 test: 8533
total UNK is  14349
sum of p(w): 1.0000000000001779
vocab: 17227
train: 886.7326210620298
dev  : 661.3054058443886
test : 658.6142906338957
sample 1:  the once UNK and the is the the When as the UNK each in of and to the of on of two time between and The in UNK that the of
sample 2:  in with to of working of of the by the the UNK UNK serene in all one few UNK of UNK UNK the UNK UNK of to and is of first
-----------------------
reuters
	dev: reuters/reuters.dev.txt
	test: reuters/reuters.test.txt
	train: reuters/reuters.train.txt
reuters  read. train: 38169 dev: 8082 test: 8214
total UNK is  10648
sum of p(w): 0.9999999999998426
vocab: 15410
train: 1016.4380307825963
dev  : 859.134696374125
test : 856.5198628865861
sample 1:  UNK not of liquids on of the the the the said UNK government with the and and UNK mln in UNK and non did

Run Trigram with Laplace

In [253]:
dnames = [ "brown","reuters", "gutenberg"]
datas = []
models = []
# Learn the models for each of the domains, and evaluate it
for dname in dnames:
    print("-----------------------")
    print(dname)
    data = read_texts("data/corpora.tar.gz", dname)
    datas.append(data)
    model = learn_trigram(data)
    models.append(model)
# compute the perplexity of all pairs
n = len(dnames)
perp_dev = np.zeros((n,n))
perp_test = np.zeros((n,n))
perp_train = np.zeros((n,n))
for i in xrange(n):
    for j in xrange(n):
        perp_dev[i][j] = models[i].perplexity(datas[j].dev)
        perp_test[i][j] = models[i].perplexity(datas[j].test)
        perp_train[i][j] = models[i].perplexity(datas[j].train)

print("-------------------------------")
print("x train")
print_table(perp_train, dnames, dnames, "table-train.tex")
print("-------------------------------")
print("x dev")
print_table(perp_dev, dnames, dnames, "table-dev.tex")
print("-------------------------------")
print("x test")
print_table(perp_test, dnames, dnames, "table-test.tex")

-----------------------
brown
	test: brown/brown.test.txt
	train: brown/brown.train.txt
	dev: brown/brown.dev.txt
brown  read. train: 39802 dev: 8437 test: 8533
total UNK is  17763
finished processing tri with unk
finished processing bi with unk
tot is  1387366.0 v is 417.46000000000004
vocab: 578551
--------- [] ------------
train: 372.94952566479816
--------- [] ------------
dev  : 569.4867873408967
--------- [] ------------
test : 570.1312713742814
-----------------------
reuters
	dev: reuters/reuters.dev.txt
	test: reuters/reuters.test.txt
	train: reuters/reuters.train.txt
reuters  read. train: 38169 dev: 8082 test: 8214
total UNK is  14248
finished processing tri with unk
finished processing bi with unk
tot is  1930106.0 v is 360.37
vocab: 628353
--------- [] ------------
train: 208.82598428586292
--------- [] ------------
dev  : 426.0229991514343
--------- [] ------------
test : 429.4876318687648
-----------------------
gutenberg
	dev: gutenberg/gutenberg.dev.txt
	train: gutenber

In [411]:
dnames = [ "brown","reuters", "gutenberg"]
datas = []
models = []
# Learn the models for each of the domains, and evaluate it
for dname in dnames:
    print("-----------------------")
    print(dname)
    data = read_texts("data/corpora.tar.gz", dname)
    datas.append(data)
    model = learn_trigram(data)
    models.append(model)

-----------------------
brown
	test: brown/brown.test.txt
	train: brown/brown.train.txt
	dev: brown/brown.dev.txt
brown  read. train: 39802 dev: 8437 test: 8533
total UNK is  12016
finished processing tri with unk
finished processing bi with unk
-----------------------
reuters
	dev: reuters/reuters.dev.txt
	test: reuters/reuters.test.txt
	train: reuters/reuters.train.txt
reuters  read. train: 38169 dev: 8082 test: 8214
total UNK is  8865
finished processing tri with unk
finished processing bi with unk
-----------------------
gutenberg
	dev: gutenberg/gutenberg.dev.txt
	train: gutenberg/gutenberg.train.txt
	test: gutenberg/gutenberg.test.txt
gutenberg  read. train: 68740 dev: 14729 test: 14826
total UNK is  5753
finished processing tri with unk
finished processing bi with unk


In [440]:
v1 = models[0].fimodel
sortv1 = sorted(v1.items(), key=lambda kv: kv[1])
print([w[0] for w in sortv1[len(v1)-10000:len(v1)-9900]])
v2 = models[1].fimodel
sortv2 = sorted(v2.items(), key=lambda kv: kv[1])
#print([w[0] for w in sortv2[len(v2)-20:]])
v3 = models[2].fimodel
sortv3 = sorted(v3.items(), key=lambda kv: kv[1])
#print([w[0] for w in sortv3[len(v3)-20:]])

[('*', 'Dr', 'Bonnor'), ('up', 'with', 'the'), ('*', 'Miss', 'Schwarzkopf'), ('the', 'spirit', 'world'), ('that', 'no', 'matter'), ('and', 'at', 'times'), ('should', 'not', 'perish'), ('is', 'only', 'one'), ('see', 'the', 'light'), ('take', 'the', 'sacred'), ('New', 'England', 'Congregational'), ('New', 'England', 'theology'), ('of', 'three', 'hundred'), ('UNK', 'them', 'and'), ('and', 'had', 'the'), ('the', 'ground', 'and'), ('of', 'many', 'factors'), ('to', 'show', 'your'), ('to', 'show', 'the'), ('the', 'East', 'END_OF_SENTENCE'), ('power', 'of', 'UNK'), ('as', 'many', 'other'), ('UNK', 'them', 'to'), ('with', 'him', 'for'), ('two', 'years', 'in'), ('the', 'farm', 'and'), ('UNK', 'its', 'UNK'), ('because', 'he', 'knew'), ('they', 'are', 'UNK'), ('was', 'UNK', 'the'), ('they', 'are', 'not'), ('they', 'are', 'the'), ('they', 'are', 'in'), ('was', 'UNK', 'with'), ('000', '000', 'the'), ('University', 'of', 'Oklahoma'), ('He', 'has', 'served'), ('the', 'State', 'Board'), ('to', 'meet', 

In [437]:
models[0].logprob_sentence(['*','If','you','is','one','of','member','of','the','United','States'],datas[0].train)

-65.335927830711

In [441]:
models[0].logprob_sentence(['*','Dr','Bonner','is','one','of','member','of','University','of','Oklahoma','in','the','past'],datas[0].train)

-94.83004760995267

In [ ]:
models[0].logprob_sentence(['*','If','you','is','one','of','member','of','the','United','States'],datas[0].train)

In [433]:
v1 = models[0].fimodelbi
sortv1 = sorted(v1.items(), key=lambda kv: kv[1])
print([w[0] for w in sortv1[len(v1)-20:]])
v2 = models[1].fimodelbi
sortv2 = sorted(v2.items(), key=lambda kv: kv[1])
print([w[0] for w in sortv2[len(v2)-20:]])
v3 = models[2].fimodelbi
sortv3 = sorted(v3.items(), key=lambda kv: kv[1])
print([w[0] for w in sortv3[len(v3)-20:]])

[('through', 'the'), ('does', 'not'), ('may', 'be'), ('rather', 'than'), ('will', 'be'), ('kind', 'of'), ('per', 'cent'), ('from', 'the'), ('should', 'be'), ('during', 'the'), ('in', 'the'), ('part', 'of'), ('able', 'to'), ('of', 'the'), ('at', 'the'), ('It', 'is'), ('New', 'York'), ('on', 'the'), ('number', 'of'), ('United', 'States')]
[('added', 'END_OF_SENTENCE'), ('4TH', 'QTR'), ('He', 'said'), ('sources', 'said'), ('lt', 'UNK'), ('QTR', 'NET'), ('mths', 'Shr'), ('compared', 'with'), ('expected', 'to'), ('due', 'to'), ('did', 'not'), ('United', 'States'), ('CORP', 'lt'), ('Nine', 'mths'), ('09', '87'), ('NET', 'Shr'), ('INC', 'lt'), ('1ST', 'QTR'), ('3RD', 'QTR'), ('Avg', 'shrs')]
[('dare', 'say'), ('ark', 'of'), ('sort', 'of'), ('inhabitants', 'of'), ('obliged', 'to'), ('pray', 'thee'), ('round', 'about'), ('Frank', 'Churchill'), ('midst', 'of'), ('into', 'the'), ('Captain', 'Wentworth'), ('spake', 'unto'), ('out', 'of'), ('Oh', 'END_OF_SENTENCE'), ('sons', 'of'), ('tribe', 'of'),

In [384]:
#!/bin/python

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import collections
from math import log
import sys
from nltk.util import ngrams

# Python 3 backwards compatibility tricks
if sys.version_info.major > 2:

    def xrange(*args, **kwargs):
        return iter(range(*args, **kwargs))

    def unicode(*args, **kwargs):
        return str(*args, **kwargs)

class LangModel:
    def fit_corpus(self, corpus):
        """Learn the language model for the whole corpus.

        The corpus consists of a list of sentences."""
        for s in corpus:
            self.fit_sentence(s)
         
        vo = self.model.keys()
        vobi = self.modelbi.keys()
        votri = self.modeltri.keys()
        for w in vo:
            if self.model[w] == 1:
                self.poplist.append(w[0])
        self.model[('UNK',)] = len(self.poplist)
        print("total UNK is ", len(self.poplist))
        for w in self.poplist:
            self.model.pop((w,))
        for tri in votri:
            old = tri
            tri = list(tri)
            for i in range(0,len(tri)):
                if tri[i] in self.poplist:
                    tri[i] = 'UNK'
            tri = tuple(tri)
            if tri in self.fimodel:
                    self.fimodel[tri] += self.modeltri[old]
            else:
                    self.fimodel[tri] = self.modeltri[old]
        print("finished processing tri with unk")    
        for bi in vobi:
            oldbi = bi
            bi = list(bi)
            for i in range(0,len(bi)):
                if bi[i] in self.poplist:
                    bi[i] = 'UNK'
            bi = tuple(bi)
            if bi in self.fimodelbi:
                self.fimodelbi[bi] += self.modelbi[oldbi]
            else:
                self.fimodelbi[bi] = self.modelbi[oldbi]   
        print("finished processing bi with unk")    
        
        self.norm(corpus)

    def perplexity(self, corpus):
        """Computes the perplexity of the corpus by the model.

        Assumes the model uses an EOS symbol at the end of each sentence.
        """
        print('---------',self.pro,'------------')
        return pow(2.0, self.entropy(corpus))

    def entropy(self, corpus):
        num_words = 0.0
        sum_logprob = 0.0
        for s in corpus:
            num_words += len(s)  # for EOS
            sum_logprob += self.logprob_sentence(s,corpus)
        return -(1.0/num_words)*(sum_logprob)

    def logprob_sentence(self, sentence,corpus):
        p = 0.0
        trigrams = list(ngrams(sentence, 3, pad_left = True, left_pad_symbol= '*', pad_right=True,right_pad_symbol='END_OF_SENTENCE'))
        trigrams = trigrams[:len(trigrams)-1]
        for i in xrange(len(sentence)):
            p += self.cond_logprob(trigrams[i],trigrams[:i])
        return p

    # required, update the model when a sentence is observed
    def fit_sentence(self, sentence): pass
    # optional, if there are any post-training steps (such as normalizing probabilities)
    def norm(self,corpus): pass
    # required, return the log2 of the conditional prob of word, given previous words
    def cond_logprob(self, word, previous): pass
    # required, the list of words the language model suports (including EOS)
    def vocab(self): pass

class Trigram(LangModel):
    def __init__(self, backoff = 0.000001):
        self.modeltri = dict()
        self.modelbi = dict()
        self.model = dict()
        self.fimodel = dict()
        self.fimodelbi = dict()
        self.lbackoff = log(backoff, 2)
        self.pro = list()
        self.corpus = list()
        self.poplist = list()
    
    def inc_word(self, w):
        opt = len(w)
        if opt == 1:
            if w in self.model:
                self.model[w] += 1.0
            else:
                self.model[w] = 1.0
        elif opt == 2:
            if w in self.modelbi:
                self.modelbi[w] += 1.0
            else:
                self.modelbi[w] = 1.0
        elif opt == 3:
            if w in self.modeltri:
                self.modeltri[w] += 1.0
            else:
                self.modeltri[w] = 1.0
        
        
    def fit_sentence(self, sentence):
        unigrams = list(ngrams(sentence, 1, pad_left = False, pad_right=False))
        bigrams = list(ngrams(sentence, 2, pad_left = True, left_pad_symbol= '*', pad_right=True,right_pad_symbol='END_OF_SENTENCE'))
        trigrams = list(ngrams(sentence, 3, pad_left = True, left_pad_symbol= '*', pad_right=True,right_pad_symbol='END_OF_SENTENCE'))
        trigrams = trigrams[:len(trigrams)-1]
        for w in unigrams:
            self.inc_word(('END_OF_SENTENCE',))
            self.inc_word(w)
        for w in bigrams:
            self.inc_word(w)
        for w in trigrams:
            self.inc_word(w)
       
    def norm(self,corpus):
        """Normalize and convert to log2-probs."""
        #binum = 0
        #for k in self.fimodelbi:
         #   if "END_OF_SENTENCE" not in k:
          #      binum += 1
        v = 0.01*len(self.model.keys())

        for word in self.fimodel:
            if (word[0],word[1]) == ('*','*') :
                self.fimodel[word] = log(0.01+self.fimodel[word],2)-log(v+len(corpus),2)
        #        self.modeltri[word] = log(1+self.modeltri[word],2)-log(v+len(corpus),2)
            else:
                self.fimodel[word] = log(0.01+self.fimodel[word], 2) -log(v+self.fimodelbi[(word[0],word[1])],2) 
        #        self.modeltri[word] = log(1+self.modeltri[word], 2) -log(v+self.modelbi[(word[0],word[1])],2)
        for word in self.fimodelbi:
            if word[0] == '*':
                self.fimodelbi[word] = log(0.01+self.fimodelbi[word],2)-log(v+len(corpus),2)
            else:
                self.fimodelbi[word] = log(0.01+self.fimodelbi[word], 2) -log(v+self.model[(word[0],)],2) 
        tot = 0
        for w in self.model:
                tot += self.model[w]
        for word in self.model:
            self.model[word] = log(0.01+self.model[word], 2) - log(v+tot, 2)
            
    def cond_logprob(self, word, previous):
        uni = self.model.keys()
        word = list(word)
        for i in range(0,len(word)):
            if (word[i] in self.poplist) or ((word[i],) not in uni):
                word[i] = 'UNK'
        word = tuple(word)
        
        if word in self.fimodel:
            qsum = 0.8*self.fimodel[word]+0.1*self.fimodelbi[(word[1],word[2])]+0.1*self.model[(word[2],)]
        elif (word[1],word[2]) in self.fimodelbi:
            qsum = 0.8*self.fimodelbi[(word[1],word[2])]+0.2*self.model[(word[2],)]
        #elif word[2] == 'UNK':
         #   qsum = 0.01*self.model[(word[2],)]
        else:
            qsum = self.model[(word[2],)]
        return qsum

    def vocab(self):
      #  return self.modeltri.keys()
        return self.fimodel.keys()

In [396]:
dnames = [ "brown","reuters", "gutenberg"]
datassss = []
modelssss = []
# Learn the models for each of the domains, and evaluate it
for dname in dnames:
    print("-----------------------")
    print(dname)
    data = read_texts("data/corpora.tar.gz", dname)
    datassss.append(data)
    model = learn_trigram(data)
    modelssss.append(model)
# compute the perplexity of all pairs
'''
n = len(dnames)
perp_dev = np.zeros((n,n))
perp_test = np.zeros((n,n))
perp_train = np.zeros((n,n))
for i in xrange(n):
    for j in xrange(n):
        perp_dev[i][j] = modelssss[i].perplexity(datas[j].dev)
        perp_test[i][j] = modelssss[i].perplexity(datas[j].test)
        perp_train[i][j] = modelssss[i].perplexity(datas[j].train)

print("-------------------------------")
print("x train")
print_table(perp_train, dnames, dnames, "table-train.tex")
print("-------------------------------")
print("x dev")
print_table(perp_dev, dnames, dnames, "table-dev.tex")
print("-------------------------------")
print("x test")
print_table(perp_test, dnames, dnames, "table-test.tex")
'''

-----------------------
brown
	test: brown/brown.test.txt
	train: brown/brown.train.txt
	dev: brown/brown.dev.txt
brown  read. train: 39802 dev: 8437 test: 8533
total UNK is  12016
finished processing tri with unk
finished processing bi with unk
-----------------------
reuters
	dev: reuters/reuters.dev.txt
	test: reuters/reuters.test.txt
	train: reuters/reuters.train.txt
reuters  read. train: 38169 dev: 8082 test: 8214
total UNK is  8865
finished processing tri with unk
finished processing bi with unk
-----------------------
gutenberg
	dev: gutenberg/gutenberg.dev.txt
	train: gutenberg/gutenberg.train.txt
	test: gutenberg/gutenberg.test.txt
gutenberg  read. train: 68740 dev: 14729 test: 14826
total UNK is  5753
finished processing tri with unk
finished processing bi with unk


'\nn = len(dnames)\nperp_dev = np.zeros((n,n))\nperp_test = np.zeros((n,n))\nperp_train = np.zeros((n,n))\nfor i in xrange(n):\n    for j in xrange(n):\n        perp_dev[i][j] = modelssss[i].perplexity(datas[j].dev)\n        perp_test[i][j] = modelssss[i].perplexity(datas[j].test)\n        perp_train[i][j] = modelssss[i].perplexity(datas[j].train)\n\nprint("-------------------------------")\nprint("x train")\nprint_table(perp_train, dnames, dnames, "table-train.tex")\nprint("-------------------------------")\nprint("x dev")\nprint_table(perp_dev, dnames, dnames, "table-dev.tex")\nprint("-------------------------------")\nprint("x test")\nprint_table(perp_test, dnames, dnames, "table-test.tex")\n'